In [18]:
### PASO 0: Instalación de librerías/bibliotecas

!pip install transformers pdfplumber

In [19]:
### PASO 1: Carga de archivo PDF

import pdfplumber

ruta_pdf = '/content/drive/MyDrive/Portafolio/Resumen_PDFs_BART/Ley31740.pdf'

if not ruta_pdf.lower().endswith('.pdf'):
    raise ValueError("El archivo no es un PDF.")

In [24]:
### PASO 2: Extracción y limpieza de texto en PDF

texto_paginas = []

with pdfplumber.open(ruta_pdf) as pdf:
    for pagina in pdf.pages:
        ancho = pagina.width
        alto = pagina.height

        columna_izquierda = pagina.within_bbox((0, 0, ancho / 2, alto))
        texto_columna_izquierda = columna_izquierda.extract_text()

        columna_derecha = pagina.within_bbox((ancho / 2, 0, ancho, alto))
        texto_columna_derecha = columna_derecha.extract_text()

        if texto_columna_izquierda and texto_columna_derecha:
            texto_consolidado = f"{texto_columna_izquierda}\n{texto_columna_derecha}"
        elif texto_columna_izquierda:
            texto_consolidado = texto_columna_izquierda
        elif texto_columna_derecha:
            texto_consolidado = texto_columna_derecha

        texto_paginas.append(texto_consolidado)

texto = "\n".join(texto_paginas)

encabezado_pdf = ["El Peruano / Sábado 13 de mayo de 2023 NORMA", "LEGALES Sábado 13 de mayo de 2023/ El Peruano"]

for encabezado in encabezado_pdf:
    texto = texto.replace(encabezado, "")

texto = texto.replace("\n", " ")

print(texto)

4 NORMAS PODER LEGISLATIVO CONGRESO DE LA REPÚBLICA LEY Nº 31740 EL PRESIDENTE DEL CONGRESO DE LA REPÚBLICA POR CUANTO: EL CONGRESO DE LA REPÚBLICA; Ha dado la Ley siguiente: LEY QUE MODIFICA LA LEY 30424, LEY QUE REGULA LA RESPONSABILIDAD ADMINISTRATIVA DE LAS PERSONAS JURÍDICAS, PARA FORTALECER LA NORMATIVA ANTICORRUPCIÓN REFERIDA A LAS PERSONAS JURÍDICAS Y PROMOVER EL BUEN GOBIERNO CORPORATIVO Artículo 1.- Modificación de la Ley 30424, Ley que regula la responsabilidad administrativa de las personas jurídicas Se modifican el título y los artículos 1, 2, 3, 5, 7, 12, 17 y 18 de la Ley 30424, Ley que regula la responsabilidad administrativa de las personas jurídicas, en los siguientes términos: “LEY QUE REGULA LA RESPONSABILIDAD ADMINISTRATIVA DE LAS PERSONAS JURÍDICAS EN EL PROCESO PENAL Artículo 1. Objeto de la Ley La presente ley regula la responsabilidad administrativa de las personas jurídicas nacionales o extranjeras en el proceso penal por los delitos previstos en los artículos

In [25]:
### PASO 3: Tokenización de textos

from transformers import AutoTokenizer

tokenizador = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
tokens = tokenizador.encode(texto, return_tensors='pt')

numero_de_tokens = tokens.shape[1]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (3576 > 512). Running this sequence through the model will result in indexing errors


In [27]:
### PASO 4: Fragmentación de tokens

from transformers import AutoTokenizer

tokenizador = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")

tokens = tokenizador.encode(texto, truncation=False)
cantidad_tokens = len(tokens)
limite_tokens = 458 #Considerando el 90% del límite: 512

print("Cantidad de tokens: " + str(cantidad_tokens))

numero_fragmentos = (cantidad_tokens + limite_tokens - 1) // limite_tokens

fragmentos = []
cant_tokens_fragmentos = []

for i in range(0, cantidad_tokens, limite_tokens):
    fragmento_tokens = tokens[i:i + limite_tokens]
    fragmento = tokenizador.decode(fragmento_tokens, skip_special_tokens=True)
    fragmentos.append(fragmento)
    cant_tokens_fragmentos.append(len(fragmento_tokens))

print("Número de fragmentos necesarios:", numero_fragmentos)
for idx, (fragmento, cantidad) in enumerate(zip(fragmentos, cant_tokens_fragmentos)):
    print(f"Fragmento {idx + 1} (Cantidad de tokens: {cantidad}):\n{fragmento}\n")

Token indices sequence length is longer than the specified maximum sequence length for this model (3576 > 512). Running this sequence through the model will result in indexing errors


Cantidad de tokens: 3576
Número de fragmentos necesarios: 8
Fragmento 1 (Cantidad de tokens: 458):
4 NORMAS PODER LEGISLATIVO CONGRESO DE LA REPÚBLICA LEY No 31740 EL PRESIDENTE DEL CONGRESO DE LA REPÚBLICA POR CUANTO: EL CONGRESO DE LA REPÚBLICA; Ha dado la Ley siguiente: LEY QUE MODIFICA LA LEY 30424, LEY QUE REGULA LA RESPONSABILIDAD ADMINISTRATIVA DE LAS PERSONAS JURÍDICAS, PARA FORTALECER LA NORMATIVA ANTICORRUPCIÓN REFERIDA A LAS PERSONAS JURÍDICAS Y PROMOVER EL BUEN GOBIERNO CORPORATIVO Artículo 1.- Modificación de la Ley 30424, Ley que regula la responsabilidad administrativa de las personas jurídicas Se modifican el título y los artículos 1, 2, 3, 5, 7, 12, 17 y 18 de la Ley 30424, Ley que regula la responsabilidad administrativa de las personas jurídicas, en los siguientes términos: “LEY QUE REGULA LA RESPONSABILIDAD ADMINISTRATIVA DE LAS PERSONAS JURÍDICAS EN EL PROCESO PENAL Artículo 1. Objeto de la Ley La presente ley regula la responsabilidad administrativa de las persona

In [28]:
### PASO 5: Traducción de fragmentos a inglés (idioma principal de BART)

from transformers import pipeline

traductor = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

fragmentos_eng = []
i = 1
for fragmento in fragmentos:
    traduccion = traductor(fragmento, max_length=3072)
    print("Traducción " + str(i) + ":")
    i += 1
    print(traduccion[0]['translation_text'])
    print("\n")
    fragmentos_eng.append(traduccion[0]['translation_text'])

print(fragmentos_eng)

Traducción 1:
4 NORMAS PODER LEGISLATIVE CONGRESS OF THE REPUBLIC LAW No. 31740 THE PRESIDENT OF THE CONGRESS OF THE REPUBLIC BY QUANTITY: THE CONGRESS OF THE REPUBLIC; It has given the following law: LAW AMENDING THE LAW 30424, LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS, TO STRENGTHENING THE ANTI-RUPTION NORMATIVE REFERRED TO THE LEGAL PERSONS AND PROMOTING THE BUEN CORPORATE GOVERNMENT Article 1.- AMENDING THE LAW 30424, LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS The title and articles 1, 2, 5, 12, 17 and 18 of the Law 30424, the Law Regulating the Administrative Liability of the Legal Persons, in the following terms: “THE LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS ARE AMENDED IN THE PENAL PROCES Article 1. Object of the Law The present law


Traducción 2:
Legal persons involved in the offences not covered by this article. Article 2. Subjective scope of application For the purposes of this law, legal per

In [29]:
### PASO 6: Fragmentación de texto en inglés

texto_ingles = " ".join(fragmentos_eng)

tokenizador = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

limite_tokens = 920 #Considerando el 90% del límite: 1024

tokens = tokenizador.encode(texto_ingles, return_tensors='pt')
cantidad_tokens = len(tokens[0])

numero_fragmentos = (cantidad_tokens + limite_tokens - 1) // limite_tokens

print("Cantidad total de tokens:", cantidad_tokens)

fragmentos_eng = []
cant_tokens_fragmentos = []

for i in range(0, cantidad_tokens, limite_tokens):
    fragmento_tokens = tokens[0][i:i + limite_tokens]
    fragmento = tokenizador.decode(fragmento_tokens, skip_special_tokens=True)
    fragmentos_eng.append(fragmento)
    cant_tokens_fragmentos.append(len(fragmento_tokens))

print("Número de fragmentos necesarios:", numero_fragmentos)
for idx, (fragmento, cantidad) in enumerate(zip(fragmentos_eng, cant_tokens_fragmentos)):
    print(f"Fragmento {idx + 1} (Cantidad de tokens: {cantidad}):\n{fragmento}\n")

Cantidad total de tokens: 1876
Número de fragmentos necesarios: 3
Fragmento 1 (Cantidad de tokens: 920):
4 NORMAS PODER LEGISLATIVE CONGRESS OF THE REPUBLIC LAW No. 31740 THE PRESIDENT OF THE CONGRESS OF THE REPUBLIC BY QUANTITY: THE CONGRESS OF THE REPUBLIC; It has given the following law: LAW AMENDING THE LAW 30424, LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS, TO STRENGTHENING THE ANTI-RUPTION NORMATIVE REFERRED TO THE LEGAL PERSONS AND PROMOTING THE BUEN CORPORATE GOVERNMENT Article 1.- AMENDING THE LAW 30424, LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS The title and articles 1, 2, 5, 12, 17 and 18 of the Law 30424, the Law Regulating the Administrative Liability of the Legal Persons, in the following terms: “THE LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS ARE AMENDED IN THE PENAL PROCES Article 1. Object of the Law The present law Legal persons involved in the offences not covered by this article. Article

In [30]:
### PASO 7: Resumen de texto con modelo BART

modelo_resumen = pipeline("summarization", model="facebook/bart-large-cnn")

resumen_total = ""

for fragmento in fragmentos_eng:
    prompt = f"Por favor, resume el siguiente texto, asegurándote de incluir detalles de la ley, sanciones/multas o posibles implementaciones:\n\n{fragmento}"
    resumen = modelo_resumen(prompt, fragmento, max_length=350, min_length=150, do_sample=False)
    resumen_total += resumen[0]['summary_text'] + "\n\n"

print("\nResumen total del texto:\n")
print(resumen_total)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Ignoring args : ('4 NORMAS PODER LEGISLATIVE CONGRESS OF THE REPUBLIC LAW No. 31740 THE PRESIDENT OF THE CONGRESS OF THE REPUBLIC BY QUANTITY: THE CONGRESS OF THE REPUBLIC; It has given the following law: LAW AMENDING THE LAW 30424, LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS, TO STRENGTHENING THE ANTI-RUPTION NORMATIVE REFERRED TO THE LEGAL PERSONS AND PROMOTING THE BUEN CORPORATE GOVERNMENT Article 1.- AMENDING THE LAW 30424, LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS The title and articles 1, 2, 5, 12, 17 and 18 of the Law 30424, the Law Regulating the Administrative Liability of the Legal Persons, in the following terms: “THE LAW REGULATING THE ADMINISTRATIVE RESPONSIBILITY OF THE LEGAL PERSONS ARE AMENDED IN THE PENAL PROCES Article 1. Object of the Law The present law Legal persons involved in the offences not covered by this article. Article 2. Subjective scope of application For the purposes of this law, legal persons are ent


Resumen total del texto:

The law was passed on 13 May, 2023. It is aimed at strengthening the Anti-RUPTION NORMATIVE. It applies to legal persons, associations, foundations, non-governmental organizations, irregular companies, entities administering an autonomous estate and enterprises of the Peruvian State. The fine is no less than ten (10) or more than ten thousand (10 000) tax units (ITU). The judges impose the fine, in a reasoned manner, according to the following criteria of substantiation and determination, as appropriate. The change of name, name or social reason, corporate reorganization, transformation, division, merger, liquidation or any act that may affect the legal personality of the entity does not prevent the attribution of responsibility to the same entity.

The fine must be paid within the ten working days of the judgment that is consented or enforceable. The judge only imposes the administrative measure of fine, which can be reduced by up to ninety percent. Confessi